In [1]:
import sys
import os
project_dir = os.path.dirname(os.getcwd())
sys.path.append(project_dir) #notebooks can't look into source unless you append the parent project directory to the path
print(project_dir)

/home/ubuntu/courtney/usal_echo


In [2]:
sys.path.append('../src')
from d00_utils.db_utils import *

/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
io_clean = dbReadWriteClean()

In [4]:
# 1. Import a_modvolume, a_measgraphref, a_measgraphic,
#        measurement_abstract_rpt
a_modvolume_df = io_clean.get_table("a_modvolume")
a_measgraphref_df = io_clean.get_table("a_measgraphref")
a_measgraphic_df = io_clean.get_table("a_measgraphic")
measurement_abstract_rpt_df = io_clean.get_table("measurement_abstract_rpt")

/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
a_measgraphref_df['studyidk_measabstractnumber_meassequence'] = a_measgraphref_df[['studyidk', 'measabstractnumber', 'meassequence']].astype(str).apply(lambda x: '_'.join(x), axis=1)

KeyboardInterrupt: 

In [6]:
print('a_measgraphref shape: {}'.format(a_measgraphref_df.shape))
print('a_measgraphref unique instance ids: {}'.format(a_measgraphref_df['instanceidk'].nunique()))
print('a_measgraphref unique study ids: {}'.format(a_measgraphref_df['studyidk'].nunique()))

print('unique study and measabract numbers: {}'.format(a_measgraphref_df['studyidk_measabstractnumber_meassequence'].nunique()))
a_measgraphref_df.head()

a_measgraphref shape: (858255, 11)
a_measgraphref unique instance ids: 254169
a_measgraphref unique study ids: 24617
unique study and measabract numbers: 858255


,studyidk,measabstractnumber,meassequence,instanceidk,indexinmglist,value,howentered,srinstanceidk,imagesopinstanceuid,measurementuid,studyidk_measabstractnumber_meassequence
0,2882,187,0,57164,0,28.242271,0,,,,2882_187_0
1,2882,188,0,57164,0,7.897812,0,,,,2882_188_0
2,2882,189,0,57164,0,85.725750,0,,,,2882_189_0
3,2882,190,0,57164,0,87.190031,0,,,,2882_190_0
4,2882,191,0,57164,1,36.654466,0,,,,2882_191_0


In [7]:
print('measurement_abstract_rpt_df shape: {}'.format(measurement_abstract_rpt_df.shape))
print('measurement_abstract_rpt_df unique study ids: {}'.format(measurement_abstract_rpt_df['studyidk'].nunique()))
measurement_abstract_rpt_df.head()

measurement_abstract_rpt_df shape: (1568351, 5)
measurement_abstract_rpt_df unique study ids: 24912


,studyidk,measabstractnumber,name,value,unitname
0,2882,2,(P1/2t)AVM,2.491754,cm2
1,2882,12,Area TSVI,3.606348,cm2
2,2882,43,FE(el-ps4),37.032087,%
3,2882,46,FE(MDD-ps4),35.095355,%
4,2882,125,P1/2t VM,88.291224,ms


Here, I am merging the table that captures the sequence of the measurements, with the table displaying the final measurements?  Works fine, get same number as left

Can i merge on value? If so, only 725,848 merge.  Maybe these are the final measurements?

In [8]:
# 2. merge a_measgraphref_df, measurement_abstract_rpt_df
df = pd.merge(
    a_measgraphref_df,
    measurement_abstract_rpt_df,
    how="left",
    on=["studyidk", "measabstractnumber"],
)

In [9]:
print(df.shape)
df.count()

(858255, 14)


studyidk                                    858255
measabstractnumber                          858255
meassequence                                858255
instanceidk                                 858255
indexinmglist                               858255
value_x                                     858255
howentered                                  858255
srinstanceidk                               858255
imagesopinstanceuid                         858255
measurementuid                              858255
studyidk_measabstractnumber_meassequence    858255
name                                        858255
value_y                                     858255
unitname                                    858255
dtype: int64

In [10]:
# Add a column where value_x doesn't match value_y

df['diff_values'] = df['value_x'] - df['value_y']
df['diff_values'].astype(bool).sum()

132771

I am going to assume that the a_measgraphref_df has all the measurements, and the measurement_abstract_rpt_df has the final measurements.

In [11]:
# Drop the non final measurements :value_x

df.drop(['value_y'], axis=1)

,studyidk,measabstractnumber,meassequence,instanceidk,indexinmglist,value_x,howentered,srinstanceidk,imagesopinstanceuid,measurementuid,studyidk_measabstractnumber_meassequence,name,unitname,diff_values
0,2882,187,0,57164,0,28.242271,0,,,,2882_187_0,AVIts ap4,cm2,0.0
1,2882,188,0,57164,0,7.897812,0,,,,2882_188_0,DVIts ap4,cm,0.0
2,2882,189,0,57164,0,85.725750,0,,,,2882_189_0,VTS(el-ps4),mL,0.0
3,2882,190,0,57164,0,87.190031,0,,,,2882_190_0,VTS(MDD-ps4),mL,0.0
4,2882,191,0,57164,1,36.654466,0,,,,2882_191_0,AVItd ap4,cm2,0.0
5,2882,192,0,57164,1,8.376849,0,,,,2882_192_0,DVItd ap4,cm,0.0
6,2882,193,0,57164,1,136.141959,0,,,,2882_193_0,VTD(el-ps4),mL,0.0
7,2882,194,0,57164,1,134.335580,0,,,,2882_194_0,VTD(MDD-ps4),mL,0.0
8,2882,195,0,57162,0,2.142836,0,,,,2882_195_0,Diám TSVI,cm,0.0
9,2882,196,0,57166,0,23.679518,0,,,,2882_196_0,GP med Ao,mm[Hg],0.0


In [12]:
df.shape

(858255, 15)

In [13]:
#Now to join with measuregraphic on instance idk and indeximhglist
print(a_measgraphic_df.shape)
a_measgraphic_df.head()

(486884, 8)


,instanceidk,indexinmglist,frame,graphictoolidk,longaxisindex,measidk,loopidk,instancerecordtype
0,57160,0,0,1,-1,0,0,0
1,57160,1,0,2,-1,0,0,0
2,57160,2,0,1,-1,0,0,0
3,57160,3,0,2,-1,0,0,0
4,57162,0,5,2,-1,0,0,0


In [14]:
df_2 = pd.merge(df, a_measgraphic_df, how="left", on=["instanceidk", "indexinmglist"])
df_2.count()

studyidk                                    858255
measabstractnumber                          858255
meassequence                                858255
instanceidk                                 858255
indexinmglist                               858255
value_x                                     858255
howentered                                  858255
srinstanceidk                               858255
imagesopinstanceuid                         858255
measurementuid                              858255
studyidk_measabstractnumber_meassequence    858255
name                                        858255
value_y                                     858255
unitname                                    858255
diff_values                                 858255
frame                                       823833
graphictoolidk                              823833
longaxisindex                               823833
measidk                                     823833
loopidk                        

In [15]:
print('{} measurements without frame measures'.format(858255-823833))

34422 measurements without frame measures


In [16]:
#create a dictionary to link measurements to views

measurement_2_view_dict = {'AVItd ap4' :'A4C', 'DVItd ap4' : 'A4C', 'VTD(el-ps4)': 'A4C', 'VTD(MDD-ps4)': 'A4C', 
                           'AVIts ap4': 'A4C', 'DVIts ap4':'A4C', 'VTS(el-ps4)':'A4C', 'VTS(MDD-ps4)':'A4C', 
                           'AVItd ap2':'A2C', 'DVItd ap2':'A2C', 'VTD(el-ps2)':'A2C', 'VTD(MDD-ps2)':'A2C', 
                           'AVIts ap2':'A2C', 'DVIts ap2':'A2C', 'VTS(el-ps2)': 'A2C', 'VTS(MDD-ps2)':'A2C', 
                           'Vol. AI (MOD-sp4)':'A4C', 'Vol. AI (MOD-sp2)':'A2C'}

measurement_2_chamber_dict = {'AVItd ap4' :'lv', 'DVItd ap4' : 'lv', 'VTD(el-ps4)': 'lv', 'VTD(MDD-ps4)': 'lv', 
                           'AVIts ap4': 'lv', 'DVIts ap4':'lv', 'VTS(el-ps4)':'lv', 'VTS(MDD-ps4)':'lv', 
                           'AVItd ap2':'lv', 'DVItd ap2':'lv', 'VTD(el-ps2)':'lv', 'VTD(MDD-ps2)':'lv', 
                           'AVIts ap2':'lv', 'DVIts ap2':'lv', 'VTS(el-ps2)': 'lv', 'VTS(MDD-ps2)':'lv', 
                           'Vol. AI (MOD-sp4)':'la', 'Vol. AI (MOD-sp2)':'la'}

measurement_2_cardio_moment_dict = {'AVItd ap4' :'ED', 'DVItd ap4' : 'ED', 'VTD(el-ps4)': 'ED', 'VTD(MDD-ps4)': 'ED', 
                           'AVIts ap4': 'ES', 'DVIts ap4':'ES', 'VTS(el-ps4)':'ES', 'VTS(MDD-ps4)':'ES', 
                           'AVItd ap2':'ED', 'DVItd ap2':'ED', 'VTD(el-ps2)':'ED', 'VTD(MDD-ps2)':'ED', 
                           'AVIts ap2':'ES', 'DVIts ap2':'ES', 'VTS(el-ps2)': 'ES', 'VTS(MDD-ps2)':'ES', 
                           'Vol. AI (MOD-sp4)':'ES', 'Vol. AI (MOD-sp2)':'ES'}


In [17]:
# Add these dictionaries to the df_2

df_2['view'] = df['name'].map(measurement_2_view_dict)
df_2['chamber'] = df['name'].map(measurement_2_chamber_dict)
df_2['cardio_moment'] = df['name'].map(measurement_2_cardio_moment_dict)
df_2.head()

,studyidk,measabstractnumber,meassequence,instanceidk,indexinmglist,value_x,howentered,srinstanceidk,imagesopinstanceuid,measurementuid,...,diff_values,frame,graphictoolidk,longaxisindex,measidk,loopidk,instancerecordtype,view,chamber,cardio_moment
0,2882,187,0,57164,0,28.242271,0,,,,...,0.0,8.0,5.0,-1.0,0.0,0.0,0.0,A4C,lv,ES
1,2882,188,0,57164,0,7.897812,0,,,,...,0.0,8.0,5.0,-1.0,0.0,0.0,0.0,A4C,lv,ES
2,2882,189,0,57164,0,85.725750,0,,,,...,0.0,8.0,5.0,-1.0,0.0,0.0,0.0,A4C,lv,ES
3,2882,190,0,57164,0,87.190031,0,,,,...,0.0,8.0,5.0,-1.0,0.0,0.0,0.0,A4C,lv,ES
4,2882,191,0,57164,1,36.654466,0,,,,...,0.0,15.0,5.0,-1.0,0.0,0.0,0.0,A4C,lv,ED


In [18]:
print(df_2['view'].value_counts())
print(df_2['chamber'].value_counts())
print(df_2['cardio_moment'].value_counts())

A4C    152251
A2C     83678
Name: view, dtype: int64
lv    216044
la     19885
Name: chamber, dtype: int64
ES    126732
ED    109197
Name: cardio_moment, dtype: int64


In [19]:
df_2.count()

studyidk                                    858255
measabstractnumber                          858255
meassequence                                858255
instanceidk                                 858255
indexinmglist                               858255
value_x                                     858255
howentered                                  858255
srinstanceidk                               858255
imagesopinstanceuid                         858255
measurementuid                              858255
studyidk_measabstractnumber_meassequence    858255
name                                        858255
value_y                                     858255
unitname                                    858255
diff_values                                 858255
frame                                       823833
graphictoolidk                              823833
longaxisindex                               823833
measidk                                     823833
loopidk                        

In [21]:
# Cut out the rows without view, chamber and cadio_moment

df_3 = df_2[pd.notna(df_2['view']) == True]
df_3.shape

(235929, 24)

In [22]:
print('Unique Study IDs: {}'.format(df_3['studyidk'].nunique()))
print('Unique Instance Ids: {}'.format(df_3['instanceidk'].nunique()))
print('Unique meaurement sequences: {}'.format(df_3['studyidk_measabstractnumber_meassequence'].nunique()))
df_3.head(3)

Unique Study IDs: 18628
Unique Instance Ids: 32065
Unique meaurement sequences: 235929


,studyidk,measabstractnumber,meassequence,instanceidk,indexinmglist,value_x,howentered,srinstanceidk,imagesopinstanceuid,measurementuid,...,diff_values,frame,graphictoolidk,longaxisindex,measidk,loopidk,instancerecordtype,view,chamber,cardio_moment
0,2882,187,0,57164,0,28.242271,0,,,,...,0.0,8.0,5.0,-1.0,0.0,0.0,0.0,A4C,lv,ES
1,2882,188,0,57164,0,7.897812,0,,,,...,0.0,8.0,5.0,-1.0,0.0,0.0,0.0,A4C,lv,ES
2,2882,189,0,57164,0,85.725750,0,,,,...,0.0,8.0,5.0,-1.0,0.0,0.0,0.0,A4C,lv,ES


In [23]:
df_3.columns

Index(['studyidk', 'measabstractnumber', 'meassequence', 'instanceidk',
       'indexinmglist', 'value_x', 'howentered', 'srinstanceidk',
       'imagesopinstanceuid', 'measurementuid',
       'studyidk_measabstractnumber_meassequence', 'name', 'value_y',
       'unitname', 'diff_values', 'frame', 'graphictoolidk', 'longaxisindex',
       'measidk', 'loopidk', 'instancerecordtype', 'view', 'chamber',
       'cardio_moment'],
      dtype='object')

In [24]:
# dropping -1 instances
print(df_3.shape)
df_3_revised = df_3[df_3['instanceidk'] != -1]
print(df_3_revised.shape)

(235929, 24)
(230290, 24)


Only 235,929 relevant measurements for our segment masks - cut the rest?

Purpose of the table is to get a ground thruth of study_id, instance_id, frame_no, view, chamber, cardio_moment

Which traces to the chords...in a_modVolume

In [25]:
df_4 = df_3_revised[['studyidk', 'indexinmglist', 'instanceidk', 'frame','view',
       'chamber', 'cardio_moment']].copy()
print(df_4.shape)
df_4 = df_4.drop_duplicates()
print(df_4.shape)

(230290, 7)
(72487, 7)


In [26]:
#Create a instanceidk_indeximglist code
df_4['instanceidk_indeximglist'] = df_4[['instanceidk', 'indexinmglist']].astype(str).apply(lambda x: '_'.join(x), axis=1)
print(df_4['instanceidk_indeximglist'].nunique())

#Create a instanceidk_indeximglist_frame code
df_4['instanceidk_indeximglist_frame'] = df_4[['instanceidk', 'indexinmglist', 'frame']].astype(str).apply(lambda x: '_'.join(x), axis=1)
print(df_4['instanceidk_indeximglist_frame'].nunique())

#Create a instanceidk_indeximglist_chamber code
df_4['instanceidk_indeximglist_chamber'] = df_4[['instanceidk', 'indexinmglist', 'chamber']].astype(str).apply(lambda x: '_'.join(x), axis=1)
print(df_4['instanceidk_indeximglist_chamber'].nunique())

#Create a instanceidk_indeximglist_view code
df_4['instanceidk_indeximglist_view'] = df_4[['instanceidk', 'indexinmglist', 'view']].astype(str).apply(lambda x: '_'.join(x), axis=1)
print(df_4['instanceidk_indeximglist_view'].nunique())

#Create a instanceidk_indeximglist_cardio_moment code
df_4['instanceidk_indeximglist_cardio_moment'] = df_4[['instanceidk', 'indexinmglist', 'cardio_moment']].astype(str).apply(lambda x: '_'.join(x), axis=1)
print(df_4['instanceidk_indeximglist_cardio_moment'].nunique())

#Create a duplicate code
df_4['duplicate'] = df_4[['studyidk', 'instanceidk', 'indexinmglist', 'frame', 'chamber', 'view', 'cardio_moment']].astype(str).apply(lambda x: '_'.join(x), axis=1)
print(df_4['duplicate'].nunique())

#Create a duplicate code
df_4['study_instance'] = df_4[['studyidk', 'instanceidk']].astype(str).apply(lambda x: '_'.join(x), axis=1)
print(df_4['study_instance'].nunique())

#Create a duplicate code
df_4['study_instance_indexinmglist'] = df_4[['studyidk', 'instanceidk', 'indexinmglist']].astype(str).apply(lambda x: '_'.join(x), axis=1)
print(df_4['study_instance_indexinmglist'].nunique())

#Create a duplicate code
df_4['instance_indexinmglist'] = df_4[['instanceidk', 'indexinmglist']].astype(str).apply(lambda x: '_'.join(x), axis=1)
print(df_4['instance_indexinmglist'].nunique())


72487
72487
72487
72487
72487
72487
32064
72487
72487


In [27]:
print(df_4.shape)
df_4.head()


(72487, 16)


,studyidk,indexinmglist,instanceidk,frame,view,chamber,cardio_moment,instanceidk_indeximglist,instanceidk_indeximglist_frame,instanceidk_indeximglist_chamber,instanceidk_indeximglist_view,instanceidk_indeximglist_cardio_moment,duplicate,study_instance,study_instance_indexinmglist,instance_indexinmglist
0,2882,0,57164,8.0,A4C,lv,ES,57164_0,57164_0_8.0,57164_0_lv,57164_0_A4C,57164_0_ES,2882_57164_0_8.0_lv_A4C_ES,2882_57164,2882_57164_0,57164_0
4,2882,1,57164,15.0,A4C,lv,ED,57164_1,57164_1_15.0,57164_1_lv,57164_1_A4C,57164_1_ED,2882_57164_1_15.0_lv_A4C_ED,2882_57164,2882_57164_1,57164_1
28,2966,0,58207,1.0,A4C,lv,ES,58207_0,58207_0_1.0,58207_0_lv,58207_0_A4C,58207_0_ES,2966_58207_0_1.0_lv_A4C_ES,2966_58207,2966_58207_0,58207_0
32,2966,1,58207,8.0,A4C,lv,ED,58207_1,58207_1_8.0,58207_1_lv,58207_1_A4C,58207_1_ED,2966_58207_1_8.0_lv_A4C_ED,2966_58207,2966_58207_1,58207_1
81,5898,0,102950,17.0,A4C,lv,ED,102950_0,102950_0_17.0,102950_0_lv,102950_0_A4C,102950_0_ED,5898_102950_0_17.0_lv_A4C_ED,5898_102950,5898_102950_0,102950_0


In [28]:
print('a_modVolume shape: {}'.format(a_modvolume_df.shape))
print('a_modVolume unique instance ids: {}'.format(a_modvolume_df['instanceidk'].nunique()))
a_modvolume_df.head()

a_modVolume shape: (1626282, 10)
a_modVolume unique instance ids: 33641


,instanceidk,indexinmglist,chordsequence,chordtype,x1coordinate,y1coordinate,chordlength,x2coordinate,y2coordinate,interchorddistance
0,57164,0,0,Eje largo,422.000000,306.000000,7.897812,297.000000,120.000000,0.394891
1,57164,0,1,Disco,408.704390,308.185087,0.762735,426.667432,296.113151,0.394891
2,57164,0,2,Disco,383.182645,311.836529,1.664436,422.381487,285.493221,0.394891
3,57164,0,3,Disco,327.646815,335.658726,3.832654,417.908971,274.998676,0.394891
4,57164,0,4,Disco,315.481865,330.333827,4.321433,417.255185,261.937779,0.394891


In [29]:
modvolume_isntanceidk_indexinmglist = a_modvolume_df[['instanceidk', 'indexinmglist']]
modvolume_isntanceidk_indexinmglist = modvolume_isntanceidk_indexinmglist.drop_duplicates()
modvolume_isntanceidk_indexinmglist.shape

(77442, 2)

In [30]:
df_6 = pd.merge(modvolume_isntanceidk_indexinmglist, df_4, how="left", on=['instanceidk', 'indexinmglist'])
df_6.count()

instanceidk                               77442
indexinmglist                             77442
studyidk                                  72481
frame                                     72481
view                                      72481
chamber                                   72481
cardio_moment                             72481
instanceidk_indeximglist                  72481
instanceidk_indeximglist_frame            72481
instanceidk_indeximglist_chamber          72481
instanceidk_indeximglist_view             72481
instanceidk_indeximglist_cardio_moment    72481
duplicate                                 72481
study_instance                            72481
study_instance_indexinmglist              72481
instance_indexinmglist                    72481
dtype: int64